In [1]:
import numpy as np
import os
import pandas as pd
from tqdm.notebook import tqdm
import copy
import sys
# sys.setrecursionlimit(10000)

In [2]:
class Graph:
    
    def __init__(self, n):
        self.n_nodes = n
        self.matrix = np.zeros((n, n), dtype="uint8")
        self.nodes = [set() for i in range(n)]
#         self.nodes = np.full(n, fill_value=set(), dtype="object")
    
    
    def add_edge(self, v, w):
        
        if v == w:
            self.matrix[v-1, v-1] = 2
            self.nodes[v-1].add(v)
            
        else:
            self.matrix[v-1, w-1] = 1
            self.matrix[w-1, v-1] = 1
            self.nodes[v-1].add(w)
            self.nodes[w-1].add(v)
    
    def get_node(self, v):
        return self.nodes[v-1]
    
    def get_lists(self):
        return self.nodes
    
    def get_node_edges(self):
        return {i+1:self.nodes[i] for i in range(self.n_nodes)}
    
    def get_matrix(self):
        return self.matrix
    
    def get_matrix_beautiful(self):
        return pd.DataFrame(self.matrix, columns=np.arange(1, self.n_nodes+1), index=np.arange(1, self.n_nodes+1))
    
    def sort_neighbors(self):
        self.nodes = [sorted(n) for n in self.nodes]

In [3]:
def open_graph_txt(filename, extra=False):
    with open(filename, "r") as f:
        lines = f.read().split("\n")
        n_nodes = int(lines[0])
        edges = [tuple(map(lambda i: int(i), line.split(" "))) for line in lines[1:-1]]
    
    graph = Graph(n_nodes)
    for v, w in edges:
        graph.add_edge(v, w)
    
    if extra:
        return graph, n_nodes, edges
    
    return graph

In [18]:
folder = "inputs"
filename = "grafo_4.txt"

path = os.path.join(folder, filename)

In [19]:
graph = open_graph_txt(path)
graph.sort_neighbors()

In [6]:
graph.nodes

[[1, 3, 16, 22, 31, 45, 46, 52, 82, 83, 84, 93],
 [2, 5, 11, 14, 19, 33, 37, 38, 49, 99],
 [1, 14, 36, 39, 52, 76, 80, 83, 85, 94, 95],
 [7, 15, 24, 26, 39, 71, 76, 84, 88, 92, 95, 98, 99],
 [2, 7, 14, 19, 23, 55, 59, 62, 65, 91, 94, 95, 98],
 [15, 22, 48, 50, 59, 62, 71, 79, 83, 91, 100],
 [4, 5, 13, 27, 33, 36, 43, 52, 56, 74, 84, 87, 90],
 [34, 40, 49, 50, 57, 58, 60, 65, 89],
 [10, 11, 15, 16, 21, 24, 46, 60, 65, 75, 89, 90, 99],
 [9, 20, 53, 72, 73, 82, 83, 90, 97],
 [2, 9, 14, 22, 23, 25, 46, 57, 80, 97],
 [13, 16, 45, 51, 55, 61, 70, 73, 76, 80, 85],
 [7, 12, 40, 48, 49, 58, 59, 81, 99],
 [2, 3, 5, 11, 16, 32, 45, 55, 64, 83],
 [4, 6, 9, 26, 34, 56, 65, 93, 94, 98, 100],
 [1, 9, 12, 14, 19, 25, 51, 60, 62, 89, 93, 94],
 [23, 27, 30, 34, 42, 61, 64, 72, 81, 82, 83, 89, 93, 94],
 [19, 28, 38, 44, 67, 69, 71, 72, 78, 86, 99],
 [2, 5, 16, 18, 27, 34, 35, 53, 69, 70, 86, 97, 100],
 [10, 22, 28, 31, 32, 42, 55, 74, 92, 96],
 [9, 29, 44, 50, 52, 58, 72, 82, 84, 94, 100],
 [1, 6, 11, 20

In [7]:
graph.get_node_edges()

{1: [1, 3, 16, 22, 31, 45, 46, 52, 82, 83, 84, 93],
 2: [2, 5, 11, 14, 19, 33, 37, 38, 49, 99],
 3: [1, 14, 36, 39, 52, 76, 80, 83, 85, 94, 95],
 4: [7, 15, 24, 26, 39, 71, 76, 84, 88, 92, 95, 98, 99],
 5: [2, 7, 14, 19, 23, 55, 59, 62, 65, 91, 94, 95, 98],
 6: [15, 22, 48, 50, 59, 62, 71, 79, 83, 91, 100],
 7: [4, 5, 13, 27, 33, 36, 43, 52, 56, 74, 84, 87, 90],
 8: [34, 40, 49, 50, 57, 58, 60, 65, 89],
 9: [10, 11, 15, 16, 21, 24, 46, 60, 65, 75, 89, 90, 99],
 10: [9, 20, 53, 72, 73, 82, 83, 90, 97],
 11: [2, 9, 14, 22, 23, 25, 46, 57, 80, 97],
 12: [13, 16, 45, 51, 55, 61, 70, 73, 76, 80, 85],
 13: [7, 12, 40, 48, 49, 58, 59, 81, 99],
 14: [2, 3, 5, 11, 16, 32, 45, 55, 64, 83],
 15: [4, 6, 9, 26, 34, 56, 65, 93, 94, 98, 100],
 16: [1, 9, 12, 14, 19, 25, 51, 60, 62, 89, 93, 94],
 17: [23, 27, 30, 34, 42, 61, 64, 72, 81, 82, 83, 89, 93, 94],
 18: [19, 28, 38, 44, 67, 69, 71, 72, 78, 86, 99],
 19: [2, 5, 16, 18, 27, 34, 35, 53, 69, 70, 86, 97, 100],
 20: [10, 22, 28, 31, 32, 42, 55, 74,

In [8]:
graph.get_lists()

[[1, 3, 16, 22, 31, 45, 46, 52, 82, 83, 84, 93],
 [2, 5, 11, 14, 19, 33, 37, 38, 49, 99],
 [1, 14, 36, 39, 52, 76, 80, 83, 85, 94, 95],
 [7, 15, 24, 26, 39, 71, 76, 84, 88, 92, 95, 98, 99],
 [2, 7, 14, 19, 23, 55, 59, 62, 65, 91, 94, 95, 98],
 [15, 22, 48, 50, 59, 62, 71, 79, 83, 91, 100],
 [4, 5, 13, 27, 33, 36, 43, 52, 56, 74, 84, 87, 90],
 [34, 40, 49, 50, 57, 58, 60, 65, 89],
 [10, 11, 15, 16, 21, 24, 46, 60, 65, 75, 89, 90, 99],
 [9, 20, 53, 72, 73, 82, 83, 90, 97],
 [2, 9, 14, 22, 23, 25, 46, 57, 80, 97],
 [13, 16, 45, 51, 55, 61, 70, 73, 76, 80, 85],
 [7, 12, 40, 48, 49, 58, 59, 81, 99],
 [2, 3, 5, 11, 16, 32, 45, 55, 64, 83],
 [4, 6, 9, 26, 34, 56, 65, 93, 94, 98, 100],
 [1, 9, 12, 14, 19, 25, 51, 60, 62, 89, 93, 94],
 [23, 27, 30, 34, 42, 61, 64, 72, 81, 82, 83, 89, 93, 94],
 [19, 28, 38, 44, 67, 69, 71, 72, 78, 86, 99],
 [2, 5, 16, 18, 27, 34, 35, 53, 69, 70, 86, 97, 100],
 [10, 22, 28, 31, 32, 42, 55, 74, 92, 96],
 [9, 29, 44, 50, 52, 58, 72, 82, 84, 94, 100],
 [1, 6, 11, 20

In [9]:
graph.get_matrix()

array([[2, 0, 1, ..., 0, 0, 0],
       [0, 2, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0]], dtype=uint8)

In [10]:
graph.get_matrix_beautiful()

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
1,2,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,2,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,1,0,0,1,1,0
5,0,1,0,0,0,0,1,0,0,0,...,1,0,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
97,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
98,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99,0,1,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [11]:
def graph_statistics(graph):
    print("Número de vértices:", graph.n_nodes)
    print("Número de arestas:", graph.get_matrix().sum()/2)
    print("Grau mínimo:", graph.get_matrix().sum(axis=0).min())
    print("Grau máximo:", graph.get_matrix().sum(axis=0).max())
    print("Grau médio:", graph.get_matrix().sum(axis=0).mean())
    print("Mediana do Grau:", np.median(graph.get_matrix().sum(axis=0)))

In [ ]:
graph_statistics(graph)

In [15]:
class DFS:
    def __init__(self, graph, root):
        self.graph = graph
        self.visited = np.zeros(graph.n_nodes, dtype="uint8")
        self.level = np.full(graph.n_nodes, fill_value=-1, dtype="int32")
        self.parent = np.full(graph.n_nodes, fill_value=-1, dtype="int32")
        self.level[root-1] = 0
        
        self.start_root(root)
    
    def start_root(self, root):
        self.stack = []
        self.stack.append(root)
    
    def search(self):
        while(len(self.stack) != 0):
            u = self.stack.pop()
            
            if not self.visited[u-1]:
                self.visited[u-1] = 1
                
                for v in self.graph.nodes[u-1][::-1]:
                    if not self.visited[v-1]:
                        self.stack.append(v)
                        self.parent[v-1] = u
                        self.level[v-1] = self.level[u-1] + 1
                        

In [20]:
dfs = DFS(graph, 1)
dfs.search()

In [ ]:
graph.nodes[13]

In [21]:
pd.DataFrame(list(zip(range(1, dfs.graph.n_nodes+1), dfs.level, dfs.parent)), columns=["node", "level", "parent"], index=np.arange(1, dfs.graph.n_nodes+1))

,node,level,parent
1,1,0,-1
2,2,257,1983
3,3,-1,-1
4,4,1222,8008
5,5,-1,-1
...,...,...,...
100050,100050,-1,-1
100051,100051,-1,-1
100052,100052,-1,-1
100053,100053,23950,98562


In [ ]:
net.show("graph.html")

In [ ]:
class BFS:
    def __init__(self, graph, root):
        self.graph = graph
        self.visited = np.zeros(graph.n_nodes, dtype="uint8")
        self.level = np.full(graph.n_nodes, fill_value=-1, dtype="int32")
        self.parent = np.full(graph.n_nodes, fill_value=-1, dtype="int32")
        
        self.level[root-1] = 0
        self.visited[root-1] = 1
        
        self.start_root(root)
        
    def start_root(self, root):
        self.queue = []
        self.queue.append(root)
        
    def search(self):
        
        while(len(self.queue)):
            v = self.queue.pop(0)
            
            for w in self.graph.nodes[v-1]:
                if v == w:
                        continue
                if not self.visited[w-1]:
                    self.visited[w-1] = 1
                    self.queue.append(w)
                    self.parent[w-1] = v
                    self.level[w-1] = self.level[v-1] + 1

In [ ]:
bfs = BFS(graph, 1)

In [ ]:
bfs.search()

In [ ]:
pd.DataFrame(list(zip(range(1, bfs.graph.n_nodes+1), bfs.level, bfs.parent)), columns=["node", "level", "parent"], index=np.arange(1, bfs.graph.n_nodes+1))

In [ ]:
net.show("graph.html")

In [ ]:
def write_parent_level_txt(filename, graph):
    with open(filename, "w") as f:
        for node in graph.nodes:
            f.write(f"Node: {node.value} -- Parent: {node.parent} -- Level: {node.level}\n")

In [ ]:
filename = "outputs/bfs_tree.txt"
write_parent_level_txt(filename, bfs.graph)

In [ ]:
class MinimumPath:
    
    def __init__(self, graph):
        self.graph = graph
        self.matrix = np.full((graph.n_nodes, graph.n_nodes), fill_value=-1, dtype="int32")
        self.run()
    
    def run(self):
        for v in tqdm(range(1, self.graph.n_nodes+1)):
            bfs = BFS(self.graph, v)
            bfs.search()
            for bfs_node_index in np.argwhere(bfs.visited == 1).reshape(-1):
                self.matrix[v-1, bfs_node_index] = bfs.level[bfs_node_index]
            del bfs
    
    def get_distance(self, u, v):
        return self.matrix[u-1, v-1]
    
    def get_diameter(self):
        return np.max(self.matrix)
    
    def get_matrix(self):
        return self.matrix
    
    def get_matrix_beautiful(self):
        return pd.DataFrame(self.matrix, columns=np.arange(1, self.graph.n_nodes+1), index=np.arange(1, self.graph.n_nodes+1))
    

In [ ]:
minpath = MinimumPath(graph)

In [ ]:
minpath.get_matrix()

In [ ]:
minpath.get_distance(1, 3)

In [ ]:
minpath.get_diameter()

In [ ]:
class Components:
    
    def __init__(self, graph):
        self.graph = graph
        self.visited = np.zeros(graph.n_nodes, dtype="uint8")
        self.components = []
        
        while np.argwhere(self.visited == 0).reshape(-1).shape[0] > 0:
            root = np.argwhere(self.visited == 0).reshape(-1)[0] + 1

            bfs = BFS(self.graph, root)
            bfs.search()
            
            bfs_visited_index = np.argwhere(bfs.visited == 1).reshape(-1)
            
            self.visited[bfs_visited_index] = 1
            self.components.append((bfs_visited_index+1).tolist())

    
    
    

In [ ]:
components = Components(graph)

In [ ]:
a = sorted(components.components, key=lambda x: len(x), reverse=True)

In [ ]:
b = [len(x) for x in a]

In [ ]:
c = list(zip(b, a))

In [ ]:
c

In [ ]:
from pyvis.network import Network

net = Network(notebook=True)

for v in range(1, graph.n_nodes+1):
    net.add_node(v, label=v)
    
for v, neighbors in enumerate(graph.nodes):
    for w in neighbors:
        net.add_edge(v+1, w)

net.show("graph.html")